In [2]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [68]:
column_names1 = ['user id','movie id','rating','timestamp']
dtype = {'userId':int, 'movieId': int, 'rating': str, 'timestamp': str}
dataset = pd.read_csv('movie_lens_dataset/ratings.csv', dtype=dtype, names=column_names1)
dataset.head() 
# dataset[dataset["movie id"]=="242"] 
# refined_dataset[refined_dataset['user id'] == user_id]

C:\Users\tanka\AppData\Local\Temp\ipykernel_119932\2312403103.py:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('movie_lens_dataset/ratings.csv', dtype=dtype, names=column_names1)


,user id,movie id,rating,timestamp
0,userId,movieId,rating,timestamp
1,1,110,1.0,1425941529
2,1,147,4.5,1425942435
3,1,858,5.0,1425941523
4,1,1221,5.0,1425941546


In [60]:
# dataset = dataset.astype({'rating':'float','timestamp':'int'})
# dataset[(dataset["rating"] == "4.0")]

,user id,movie id,rating,timestamp
0,userId,movieId,4.0,timestamp
1,1,110,4.0,1425941529
2,1,147,4.0,1425942435
3,1,858,4.0,1425941523
4,1,1221,4.0,1425941546
...,...,...,...,...
26024285,270896,58559,4.0,1257031564
26024286,270896,60069,4.0,1257032032
26024287,270896,63082,4.0,1257031764
26024288,270896,64957,4.0,1257033990


In [10]:
# ## same item id is same as movie id, item id column is renamed as movie id
# column_names1 = ['user id','movie id','rating','timestamp']
# dataset = pd.read_csv('ml-100k/u.data', sep='\t',header=None,names=column_names1)
# dataset.head()
# dataset[dataset["movie id"]==242] 


,user id,movie id,rating,timestamp
0,196,242,3,881250949
253,63,242,3,875747190
629,226,242,5,883888671
1232,154,242,3,879138235
2159,306,242,5,876503793
...,...,...,...,...
95720,305,242,5,886307828
96237,845,242,4,885409493
97046,500,242,3,891916883
99260,720,242,4,891262608


In [34]:
# d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
# column_names2 = d.split(' | ')
# column_names2

['movie id',
 'movie title',
 'release date',
 'video release date',
 'IMDb URL',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [36]:
# items_dataset = pd.read_csv('ml-100k/u.item', sep='|',header=None,names=column_names2,encoding='latin-1')
# items_dataset

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# movie_dataset = items_dataset[['movie id','movie title']]
# movie_dataset.head()
# movie_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie id     1682 non-null   int64 
 1   movie title  1682 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [33]:
movie_dataset = pd.read_csv('movie_lens_dataset/movies_metadata_processed.csv')
movie_dataset["movie title"] = movie_dataset["title"].astype(str) + " (" + movie_dataset["release_date"].astype(str).str.slice(0,4) + ")"
movie_dataset = movie_dataset[["id", "movie title"]]
movie_dataset.head()
movie_dataset.info()

C:\Users\tanka\AppData\Local\Temp\ipykernel_119932\3167501942.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_dataset = pd.read_csv('movie_lens_dataset/movies_metadata_processed.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44512 entries, 0 to 44511
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           44512 non-null  object
 1   movie title  44512 non-null  object
dtypes: object(2)
memory usage: 695.6+ KB


## Merging required datasets

In [69]:
merged_dataset = pd.merge(dataset, movie_dataset, how='inner', on='movie id')
merged_dataset.head()

,user id,movie id,rating,timestamp,movie title
0,1324,111,3.0,947273773,"Truth About Cats & Dogs, The (1996)"
1,1330,111,5.0,950848780,"Truth About Cats & Dogs, The (1996)"
2,1332,111,4.0,1161393560,"Truth About Cats & Dogs, The (1996)"
3,1337,111,5.0,1466902702,"Truth About Cats & Dogs, The (1996)"
4,1345,111,4.5,1372895188,"Truth About Cats & Dogs, The (1996)"


In [70]:
# Example of a multiple rating scenario by an user to a specific movie:
merged_dataset[(merged_dataset['movie title'] == 'Chasing Amy (1997)') & (merged_dataset['user id'] == 1361)]

# merged_dataset[(merged_dataset['movie title'] == 'Chasing Amy (1997)') & (merged_dataset['user id'] == 894)]

,user id,movie id,rating,timestamp,movie title
6830128,1361,246,5.0,953933347,Chasing Amy (1997)


In [71]:

merged_dataset["rating"] = pd.to_numeric(merged_dataset["rating"], downcast="float")
refined_dataset = merged_dataset.groupby(by=['user id','movie title'], as_index=False).agg({"rating":"mean"})
refined_dataset.head()

,user id,movie title,rating
0,1324,Dadetown (1995),4.0
1,1324,Purple Noon (1960),4.0
2,1324,Small Faces (1995),4.0
3,1324,"Truth About Cats & Dogs, The (1996)",3.0
4,1325,Event Horizon (1997),5.0


## Training KNN model to build collaborative recommender

In [72]:
# Reshaping model in such a way that each user has n-dimensional rating space where n is total number of movies
user_to_movie_df = refined_dataset.pivot(
    index='user id',
     columns='movie title',
      values='rating').fillna(0)

user_to_movie_df.head()

c:\Users\tanka\Desktop\movie-recommender-ir\.venv\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user id,,,,,,,,,,,,,,,,,,,,,
1324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# transform matrix to scipy sparse matrix
user_to_movie_sparse_df = csr_matrix(user_to_movie_df.values)
user_to_movie_sparse_df

<258365x1609 sparse matrix of type '<class 'numpy.float32'>'
	with 9948909 stored elements in Compressed Sparse Row format>

In [74]:
# Fitting KNN model to scipy sparse matrix

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_to_movie_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [75]:
# Giving Input as User id, Number of similar Users to be considered, Number of top movie we want to recommend

def new_recommender_system(user_df, n_similar_users, n_movies): #, user_to_movie_df, knn_model):
  
  print("Movie seen by the User:")
  print(list(user_df["movie title"]))
  print("")
  user_id = -1

  # def get_similar_users(user, user_to_movie_df, knn_model, n = 5):
  def get_similar_users(n = 5):

    knn_input_array = np.array([4.5 if col in movies else 0 for col in user_to_movie_df.columns])
    
    knn_input = np.asarray([knn_input_array])
    
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
    
    print("Top",n,"users who are very much similar to the User-",user_id, "are: ")
    print(" ")

    for i in range(1,len(distances[0])):
      print(i,". User:", indices[0][i]+1, "separated by distance of",distances[0][i])
    print("")
    return indices.flatten()[1:] + 1, distances.flatten()[1:]


  def filtered_movie_recommendations(n = 10):
  
    first_zero_index = np.where(mean_rating_list == 0)[0][-1]
    sortd_index = np.argsort(mean_rating_list)[::-1]
    sortd_index = sortd_index[:list(sortd_index).index(first_zero_index)]
    n = min(len(sortd_index),n)
    # movies_watched = list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])
    movies_watched = list(user_df["movie title"])
    filtered_movie_list = list(movies_list[sortd_index])
    count = 0
    final_movie_list = []
    for i in filtered_movie_list:
      if i not in movies_watched:
        count+=1
        final_movie_list.append(i)
      if count == n:
        break
    if count == 0:
      print("There are no movies left which are not seen by the input users and seen by similar users. May be increasing the number of similar users who are to be considered may give a chance of suggesting an unseen good movie.")
    else:
      print(final_movie_list)

  similar_user_list, distance_list = get_similar_users(n_similar_users)
  weightage_list = distance_list/np.sum(distance_list)
  mov_rtngs_sim_users = user_to_movie_df.values[similar_user_list]
  movies_list = user_to_movie_df.columns
  weightage_list = weightage_list[:,np.newaxis] + np.zeros(len(movies_list))
  new_rating_matrix = weightage_list*mov_rtngs_sim_users
  mean_rating_list = new_rating_matrix.sum(axis =0)
  print("")
  print("Movies recommended based on similar users are: ")
  print("")
  filtered_movie_recommendations(n_movies)

In [76]:
## create a new user row from movies they like

movies = ['Air Bud (1997)', 'Air Force One (1997)', 'Beverly Hills Ninja (1997)', 'Booty Call (1997)', 'Bulletproof (1996)', 'Conspiracy Theory (1997)', 'Fargo (1996)', 'Jack (1996)', 'Jungle2Jungle (1997)', 'Liar Liar (1997)', 'Love Jones (1997)', "McHale's Navy (1997)", 'Men in Black (1997)', 'Mimic (1997)', 'Money Talks (1997)', 'Murder at 1600 (1997)', 'Private Parts (1997)', 'Scream (1996)', 'Thin Line Between Love and Hate, A (1996)']

def new_user_from_movies(movies):
    user_id_list = [-1] * len(movies)
    rating_list = [4.5] * len(movies)
    keys = refined_dataset.columns
    values = [user_id_list, movies, rating_list]
    # dic = {key: value for key in keys for value in values}
    dic = {}
    for col_index in range(len(keys)):
        dic[keys[col_index]] = values[col_index]
    return pd.DataFrame(dic)

new_user_dataaset = new_user_from_movies(movies)
new_user_dataaset.head()
# type(new_user_dataaset["movie title"][0])

new_recommender_system(new_user_dataaset, 15,15)

Movie seen by the User:
['Air Bud (1997)', 'Air Force One (1997)', 'Beverly Hills Ninja (1997)', 'Booty Call (1997)', 'Bulletproof (1996)', 'Conspiracy Theory (1997)', 'Fargo (1996)', 'Jack (1996)', 'Jungle2Jungle (1997)', 'Liar Liar (1997)', 'Love Jones (1997)', "McHale's Navy (1997)", 'Men in Black (1997)', 'Mimic (1997)', 'Money Talks (1997)', 'Murder at 1600 (1997)', 'Private Parts (1997)', 'Scream (1996)', 'Thin Line Between Love and Hate, A (1996)']

Top 15 users who are very much similar to the User- -1 are: 
 
1 . User: 139893 separated by distance of 0.7705842661294382
2 . User: 182701 separated by distance of 0.7705842661294382
3 . User: 94798 separated by distance of 0.7705842661294382
4 . User: 51784 separated by distance of 0.7705842661294382
5 . User: 55565 separated by distance of 0.7705842661294382
6 . User: 147667 separated by distance of 0.7705842661294382
7 . User: 81004 separated by distance of 0.7705842661294382
8 . User: 252247 separated by distance of 0.770584266